In [2]:
#imports and setup
import numpy as np
import cv2 as cv
import os

In [3]:
'''loading video names into a list for easier access with absolute path,
video names can be access by calling the index of the video_lst and will contain 
the current absolute path, asumes that the video folder is in the same folder as this file'''

video_path = "videos"
full_path = os.path.abspath(video_path)

video_lst = []
v_dir = os.listdir(full_path)
for name in v_dir:
    video_lst.append(full_path+'/'+name)
print(video_lst[0]) 

/home/wolf/jku/ws2022/computer_vision/test_code/videos/video_023.mp4


In [2]:
#loading video into memory
cap = cv.VideoCapture(video_lst[3])

NameError: name 'cv' is not defined

In [1]:
'''Playing one video file'''

while cap.isOpened():
    ret, frame = cap.read()
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    
    '''cv_waitkey() defines miliseconds between frames, higher values play the video slower
    ord('q') defines a key to exit the video'''
    cv.imshow('frame',frame)
    if cv.waitKey(25) & 0xFF == ord('q'):
        break
cap.release() #after release from memory the video needs to be recaptured
cv.destroyAllWindows()

NameError: name 'cap' is not defined

In [30]:
'''Testing background substraction with morphological noise reduction
    opencv has KNN and MOG2 BGS build in'''

cap = cv.VideoCapture(video_lst[3])
backSub = cv.createBackgroundSubtractorKNN()
#backSub = cv.createBackgroundSubtractorMOG2()

#kernel for morphological noise reduction
#cv.MORPH_ELLIPSE
kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (3, 3));
while cap.isOpened():
    ret, frame = cap.read()
    if frame is None:
        break
    
    fgMask = backSub.apply(frame)
    fgMask = cv.morphologyEx(fgMask, cv.MORPH_OPEN, kernel);

    #cv.imshow('Frame', frame)
    cv.imshow('FG Mask', fgMask)
    
    if cv.waitKey(25) & 0xFF == ord('q'):
        break
    
cap.release()
cv.destroyAllWindows()

QObject::moveToThread: Current thread (0x55e6f61a4320) is not the object's thread (0x55e6f61c58a0).
Cannot move to target thread (0x55e6f61a4320)

QObject::moveToThread: Current thread (0x55e6f61a4320) is not the object's thread (0x55e6f61c58a0).
Cannot move to target thread (0x55e6f61a4320)

QObject::moveToThread: Current thread (0x55e6f61a4320) is not the object's thread (0x55e6f61c58a0).
Cannot move to target thread (0x55e6f61a4320)

QObject::moveToThread: Current thread (0x55e6f61a4320) is not the object's thread (0x55e6f61c58a0).
Cannot move to target thread (0x55e6f61a4320)

QObject::moveToThread: Current thread (0x55e6f61a4320) is not the object's thread (0x55e6f61c58a0).
Cannot move to target thread (0x55e6f61a4320)

QObject::moveToThread: Current thread (0x55e6f61a4320) is not the object's thread (0x55e6f61c58a0).
Cannot move to target thread (0x55e6f61a4320)

QObject::moveToThread: Current thread (0x55e6f61a4320) is not the object's thread (0x55e6f61c58a0).
Cannot move to tar

In [5]:
'''Testing Frame differencing code based on:
https://github.com/infoaryan/OPENCV-PYTHON-Zero-to-One-Course-Resources/blob/master/Video%2031%20-%20Frame%20Differencing/frame_differencing.py
Expanded with: blur, dilation and morphological noise reduction and contour finding'''

#video to capture
video = video_lst[3]

kernel = np.ones((17,17)) #kerne for dilation, influences the size of the contours 
kernel_blur = (3,3) #higher kernels do not work well for this data set
kernel_morph = cv.getStructuringElement(cv.MORPH_ELLIPSE, (7, 7)) #kernel for morphology
THRESHOLD = 30 #when to detect difference
VALUE = 200 #which value to assign to difference

#threshold to calculate
threshold_method = cv.THRESH_BINARY

# Compute the frame difference
def frame_diff(prev_frame, cur_frame, next_frame):
    diff_frames1 = cv.absdiff(next_frame, cur_frame)
    # Absolute difference between current frame and previous frame
    diff_frames2 = cv.absdiff(cur_frame, prev_frame)
    # Return the result of bitwise 'AND' between the above two resultant images
    #gives better result than simple substraction
    return cv.bitwise_and(diff_frames1, diff_frames2)

def get_frame(cap):
    ret, frame = cap.read()
    # Resize the image not needed
    '''frame = cv.resize(frame, None, fx=scaling_factor,
        fy=scaling_factor, interpolation=cv.INTER_AREA)'''
    return frame

#finds and draws contours over pixels if greater than threshold
#RETR_CCOMP, RETR_EXTERNAL seem to work best
def construct_contours(frame, contour_threshold):
    contours, hierarchy = cv.findContours(frame, cv.RETR_CCOMP, 
                                           cv.CHAIN_APPROX_NONE)
    '''cv.drawContours(frame, contours=contours, contourIdx=-1, 
                     color=(0, 255, 0), thickness=2, lineType=cv.LINE_AA)'''
    for contour in contours:
      if cv.contourArea(contour) < contour_threshold:
        continue
      (x, y, w, h) = cv.boundingRect(contour)
      #drawing a rectangle over the contours
      cv.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),3)


cap = cv.VideoCapture(video)
prev_frame = get_frame(cap)
cur_frame = get_frame(cap)
next_frame = get_frame(cap)

#applying blur will possibly extend with other option for preperation
def prepare_frames(frames,kernel):
    result = []
    for frame in frames:
        result.append(cv.GaussianBlur(frame, kernel, 0))
    return result[0], result[1], result[2]

# Iterating over all frames and applying difference and dilation
while True:
    frames = [prev_frame,cur_frame,next_frame]
    original = frames[1]
    #applying preprocessing
    prev_frame, cur_frame, next_frame = prepare_frames(frames, kernel_blur)
    
    #calculating difference
    frame_difference = frame_diff(prev_frame,cur_frame, next_frame)
    #applying dilation
    frame_difference = cv.dilate(frame_difference, kernel)
    #applying morphological noise reduction
    frame_difference = cv.morphologyEx(frame_difference, cv.MORPH_OPEN, kernel_morph)
    #converting into greyscale for contour finding
    frame_difference = cv.cvtColor(frame_difference, cv.COLOR_BGR2GRAY)
    #applying thresholds
    ret, frame_th = cv.threshold(frame_difference, THRESHOLD, VALUE, threshold_method)
    #finding and drawing contours on given frame with given threshold
    construct_contours(frame_th, 10)
    
    #cv.imshow("Difference", frame_difference)
    cv.imshow("After Threshold", frame_th)
    cv.imshow("Original",original)
    
    # Update the variables
    prev_frame = cur_frame
    cur_frame = next_frame
    next_frame = get_frame(cap)
    
    if cv.waitKey(25) & 0xFF == ord('q'):
        break
        
cap.release()
cv.destroyAllWindows()

QObject::moveToThread: Current thread (0x55d8a0442820) is not the object's thread (0x55d8a19ad360).
Cannot move to target thread (0x55d8a0442820)

QObject::moveToThread: Current thread (0x55d8a0442820) is not the object's thread (0x55d8a19ad360).
Cannot move to target thread (0x55d8a0442820)

QObject::moveToThread: Current thread (0x55d8a0442820) is not the object's thread (0x55d8a19ad360).
Cannot move to target thread (0x55d8a0442820)

QObject::moveToThread: Current thread (0x55d8a0442820) is not the object's thread (0x55d8a19ad360).
Cannot move to target thread (0x55d8a0442820)

QObject::moveToThread: Current thread (0x55d8a0442820) is not the object's thread (0x55d8a19ad360).
Cannot move to target thread (0x55d8a0442820)

QObject::moveToThread: Current thread (0x55d8a0442820) is not the object's thread (0x55d8a19ad360).
Cannot move to target thread (0x55d8a0442820)

QObject::moveToThread: Current thread (0x55d8a0442820) is not the object's thread (0x55d8a19ad360).
Cannot move to tar